In [ ]:
import pandas as pd


# --- 1. Load Data ---

In [32]:
try:
    race_df = pd.read_csv('csv/f1_race_data_raw.csv')
    standings_df = pd.read_csv('csv/f1_standings_data_raw.csv')
    print("Data loaded successfully.")
    print(f"Race DF shape: {race_df.shape}")
    print(f"Standings DF shape: {standings_df.shape}")
except FileNotFoundError:
    print("Error: CSV files not found. Make sure you ran the data collection script.")
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully.
Race DF shape: (1220, 12)
Standings DF shape: (198, 17)


In [33]:
print("\nInitial Race DF Info:")
race_df.info()
print("\nSample Race DF Head:")
print(race_df.head())
print("\nSample Standings DF Head:")
print(standings_df[['Season', 'position', 'points', 'wins', 'DriverId', 'constructorNames']].head())


Initial Race DF Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Season           1220 non-null   int64  
 1   Round            1220 non-null   int64  
 2   EventName        1220 non-null   object 
 3   Driver           1220 non-null   object 
 4   DriverId         1200 non-null   object 
 5   TeamId           1200 non-null   object 
 6   GridPosition     1198 non-null   float64
 7   FinishPosition   1198 non-null   float64
 8   PositionsGained  1171 non-null   float64
 9   PointsGained     1200 non-null   float64
 10  AvgLapTimeSec    1187 non-null   float64
 11  Status           1200 non-null   object 
dtypes: float64(5), int64(2), object(5)
memory usage: 114.5+ KB

Sample Race DF Head:
   Season  Round            EventName Driver  DriverId    TeamId  \
0    2020      1  Austrian Grand Prix    BOT    bottas  mercedes   
1   

# --- 2. Data Cleaning & Preparation ---

In [34]:
cols_to_check = ['FinishPosition', 'GridPosition', 'PointsGained']
original_rows = len(race_df)
race_df.dropna(subset=cols_to_check, how='any', inplace=True)
print(f"Dropped {original_rows - len(race_df)} rows with NaNs in essential columns (e.g., FinishPosition).")

Dropped 22 rows with NaNs in essential columns (e.g., FinishPosition).


In [35]:
numeric_cols = ['GridPosition', 'FinishPosition', 'PositionsGained', 'PointsGained', 'AvgLapTimeSec']
for col in numeric_cols:
    race_df[col] = pd.to_numeric(race_df[col], errors='coerce')
print("Converted numeric columns, coercing errors.")

Converted numeric columns, coercing errors.


In [36]:
max_grid_per_round = race_df.groupby(['Season', 'Round'])['GridPosition'].transform('max')
race_df['PitLaneStart'] = (race_df['GridPosition'] == 0).astype(int)

race_df['GridPositionCalc'] = race_df['GridPosition']

race_df.loc[race_df['GridPosition'] == 0, 'GridPositionCalc'] = race_df.loc[race_df['GridPosition'] == 0, 'GridPosition'].index.map(lambda idx: max_grid_per_round[idx] + 1)

print("Handled GridPosition=0 (PitLaneStart), created GridPositionCalc.")

Handled GridPosition=0 (PitLaneStart), created GridPositionCalc.


In [37]:
race_df['PositionsGained'] = race_df['GridPositionCalc'] - race_df['FinishPosition']
print("Recalculated PositionsGained.")

Recalculated PositionsGained.


In [38]:
print("Imputing NaNs in AvgLapTimeSec with race median...")
race_df['AvgLapTimeSec'] = race_df.groupby(['Season', 'Round'])['AvgLapTimeSec'].transform(lambda x: x.fillna(x.median()))
global_median_laptime = race_df['AvgLapTimeSec'].median()
race_df['AvgLapTimeSec'].fillna(global_median_laptime, inplace=True)
print(f"Remaining AvgLapTimeSec NaNs filled with global median: {global_median_laptime:.3f}")

Imputing NaNs in AvgLapTimeSec with race median...
Remaining AvgLapTimeSec NaNs filled with global median: 92.499


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\1864501172.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_df['AvgLapTimeSec'].fillna(global_median_laptime, inplace=True)


In [39]:
median_pos_gained = 0 # Or calculate race_df['PositionsGained'].median() if preferred
race_df['PositionsGained'].fillna(median_pos_gained, inplace=True)
print(f"Imputed PositionsGained NaNs with {median_pos_gained}.")

Imputed PositionsGained NaNs with 0.


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\1079114945.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_df['PositionsGained'].fillna(median_pos_gained, inplace=True)


In [40]:
median_grid_pos = race_df['GridPositionCalc'].median()
race_df['GridPositionCalc'].fillna(median_grid_pos, inplace=True)
print(f"Imputed GridPositionCalc NaNs with median: {median_grid_pos:.1f}")

Imputed GridPositionCalc NaNs with median: 11.0


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\2977549689.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_df['GridPositionCalc'].fillna(median_grid_pos, inplace=True)


In [41]:
median_finish_pos = race_df['FinishPosition'].median()
race_df['FinishPosition'].fillna(median_finish_pos, inplace=True)
print(f"Imputed FinishPosition NaNs with median: {median_finish_pos:.1f}")

Imputed FinishPosition NaNs with median: 10.0


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\48096444.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_df['FinishPosition'].fillna(median_finish_pos, inplace=True)


In [42]:
finished_statuses = ['Finished', '+1 Lap', '+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps', '+6 Laps'] # Add more if needed
race_df['DidFinish'] = race_df['Status'].apply(lambda x: 1 if x in finished_statuses else 0)
print("Created 'DidFinish' column.")

Created 'DidFinish' column.


In [43]:
race_df.sort_values(by=['Season', 'Round', 'DriverId'], inplace=True)
print("Sorted DataFrame by Season, Round, DriverId.")

Sorted DataFrame by Season, Round, DriverId.


# --- 3. Create Cumulative/Expanding Features ---

In [44]:
grouped = race_df.groupby(['Season', 'DriverId'])

race_df['CumulativePoints'] = grouped['PointsGained'].transform(lambda x: x.shift(1).expanding(min_periods=1).sum()).fillna(0)

race_df['AvgPoints'] = grouped['PointsGained'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean()).fillna(0)

race_df['AvgFinishPos'] = grouped['FinishPosition'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean()).fillna(race_df['FinishPosition'].median()) # Fill initial NaNs

race_df['AvgGridPos'] = grouped['GridPositionCalc'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean()).fillna(race_df['GridPositionCalc'].median())

race_df['AvgPosGained'] = grouped['PositionsGained'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean()).fillna(0)

race_df['RaceCount'] = grouped.cumcount()

race_df['FinishRate'] = grouped['DidFinish'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean()).fillna(1.0) # Assume 100% initially

race_df['DNFCount'] = grouped['DidFinish'].transform(lambda x: (1 - x.shift(1)).expanding(min_periods=1).sum()).fillna(0)

race_df['StdDevFinishPos'] = grouped['FinishPosition'].transform(lambda x: x.shift(1).expanding(min_periods=2).std()).fillna(0) # Need min 2 periods for std dev

race_df['Wins'] = grouped['FinishPosition'].transform(lambda x: (x.shift(1) == 1).expanding(min_periods=1).sum()).fillna(0)
race_df['Podiums'] = grouped['FinishPosition'].transform(lambda x: (x.shift(1) <= 3).expanding(min_periods=1).sum()).fillna(0)
race_df['Top10s'] = grouped['FinishPosition'].transform(lambda x: (x.shift(1) <= 10).expanding(min_periods=1).sum()).fillna(0)

print("Calculated expanding features (cumulative points, avg finish, avg grid, finish rate, etc.).")

Calculated expanding features (cumulative points, avg finish, avg grid, finish rate, etc.).


# --- 4. Create Rolling Features (Recent Form) ---

In [45]:
window_size = 5

race_df[f'PointsLast{window_size}'] = grouped['PointsGained'].transform(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).sum()).fillna(0)

race_df[f'AvgFinishLast{window_size}'] = grouped['FinishPosition'].transform(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean()).fillna(race_df['FinishPosition'].median())

race_df[f'AvgGridLast{window_size}'] = grouped['GridPositionCalc'].transform(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean()).fillna(race_df['GridPositionCalc'].median())

race_df[f'FinishRateLast{window_size}'] = grouped['DidFinish'].transform(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean()).fillna(1.0)

race_df[f'DNFCountLast{window_size}'] = grouped['DidFinish'].transform(lambda x: (1 - x.shift(1)).rolling(window=window_size, min_periods=1).sum()).fillna(0)

print(f"Calculated rolling features for last {window_size} races.")

Calculated rolling features for last 5 races.


# --- 5. Add Lagged Features (Previous Season Performance) ---

In [48]:
standings_lag = standings_df[['Season', 'DriverId', 'position', 'points', 'wins']].copy()
standings_lag.rename(columns={
    'position': 'PrevSeasonRank',
    'points': 'PrevSeasonPoints',
    'wins': 'PrevSeasonWins'
}, inplace=True)

standings_lag['Season'] = standings_lag['Season'] + 1

race_df = pd.merge(race_df, standings_lag, on=['Season', 'DriverId'], how='left')

race_df['PrevSeasonRank'].fillna(25, inplace=True)
race_df['PrevSeasonPoints'].fillna(0, inplace=True)
race_df['PrevSeasonWins'].fillna(0, inplace=True)

print("Merged previous season standings data.")

Merged previous season standings data.


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\507705280.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_df['PrevSeasonRank'].fillna(25, inplace=True)
C:\Users\domin\AppData\Local\Temp\ipykernel_34024\507705280.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

# --- 6. Define the Target Variable ---

In [49]:
target_standings = standings_df[['Season', 'DriverId', 'position', 'points']].copy()
target_standings.rename(columns={
    'position': 'FinalRank',
    'points': 'FinalPoints'
}, inplace=True)

final_df = pd.merge(race_df, target_standings, on=['Season', 'DriverId'], how='left')

final_df['FinalRank'].fillna(final_df['FinalRank'].max() + 1, inplace=True)
final_df['FinalPoints'].fillna(0, inplace=True)

print("Merged final season standings as target variables (FinalRank, FinalPoints).")

Merged final season standings as target variables (FinalRank, FinalPoints).


C:\Users\domin\AppData\Local\Temp\ipykernel_34024\4241360974.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['FinalRank'].fillna(final_df['FinalRank'].max() + 1, inplace=True)
C:\Users\domin\AppData\Local\Temp\ipykernel_34024\4241360974.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

# --- Final Check ---

In [52]:
feature_cols = [
    # Identifiers
    'Season', 'Round', 'EventName', 'DriverId', 'TeamId',
    # Raw Current Race Results (Careful about using these directly as features!)
    # 'GridPosition', 'FinishPosition', 'PositionsGained', 'PointsGained', 'AvgLapTimeSec', 'Status',
    # Base Calculated
    'RaceCount', 'DidFinish', 'PitLaneStart',
    # Cumulative Features
    'CumulativePoints', 'AvgPoints', 'AvgFinishPos', 'AvgGridPos', 'AvgPosGained',
    'FinishRate', 'DNFCount', 'StdDevFinishPos', 'Wins', 'Podiums', 'Top10s',
    # Rolling Features
    f'PointsLast{window_size}', f'AvgFinishLast{window_size}', f'AvgGridLast{window_size}',
    f'FinishRateLast{window_size}', f'DNFCountLast{window_size}',
    # Lagged Features
    'PrevSeasonRank', 'PrevSeasonPoints', 'PrevSeasonWins',
    # Target Variables
    'FinalRank', 'FinalPoints'
]
final_df = final_df[feature_cols]

print(f"Final DataFrame shape: {final_df.shape}")
print("\nFinal DataFrame Head:")
print(final_df.head())
print("\nFinal DataFrame Info:")
final_df.info()
print("\nChecking for remaining NaNs in final features:")
print(final_df.isnull().sum())

final_df.to_csv('csv/f1_engineered_features.csv', index=False)
print("\nEngineered features saved to f1_engineered_features.csv")

Final DataFrame shape: (7818, 29)

Final DataFrame Head:
   Season  Round            EventName DriverId    TeamId  RaceCount  \
0    2020      1  Austrian Grand Prix    albon  red_bull          0   
1    2020      1  Austrian Grand Prix    albon  red_bull          0   
2    2020      1  Austrian Grand Prix    albon  red_bull          0   
3    2020      1  Austrian Grand Prix   bottas  mercedes          0   
4    2020      1  Austrian Grand Prix   bottas  mercedes          0   

   DidFinish  PitLaneStart  CumulativePoints  AvgPoints  ...  PointsLast5  \
0          0             0               0.0        0.0  ...          0.0   
1          0             0               0.0        0.0  ...          0.0   
2          0             0               0.0        0.0  ...          0.0   
3          1             0               0.0        0.0  ...          0.0   
4          1             0               0.0        0.0  ...          0.0   

   AvgFinishLast5  AvgGridLast5  FinishRateLast5  DNF